In [47]:
import os
import pandas as pd
import numpy as np
import time

## 切换文件路径

In [46]:
os.chdir('C:\\Users\\12451\\Desktop\\zhihu')

In [48]:
df = pd.read_excel('zh_result.xlsx')

In [49]:
df.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户名,用户类型,签名,被关注人数,评论数
0,POTO,<p>出于好奇，我爬取了本回答下的24620条答案，从回答中抽取了部分答主的个人信息进行了分...,1552555253,1,True,1553271539,10205,陈苟,people,"&#34;不可能不是一种事实,只是一种意见.&#34;",968,868
1,wang-bo-jing-85,<p>活的，母的。不要点赞，丢人。</p><p></p>,1540024879,1,True,1540024947,3709,南阳居士,people,我就是我，自己看了都上火。,758,431


## 把代表性别的0，1，-1替换成女，男，未知

In [50]:
df['sex'] = np.where(df['性别'] == 1,'男',
                    np.where(df['性别'] == 0,'女','未知'))

In [8]:
df.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户名,用户类型,签名,被关注人数,评论数,sex
0,POTO,<p>出于好奇，我爬取了本回答下的24620条答案，从回答中抽取了部分答主的个人信息进行了分...,1552555253,1,True,1553271539,10205,陈苟,people,"&#34;不可能不是一种事实,只是一种意见.&#34;",968,868,男
1,wang-bo-jing-85,<p>活的，母的。不要点赞，丢人。</p><p></p>,1540024879,1,True,1540024947,3709,南阳居士,people,我就是我，自己看了都上火。,758,431,男


## 绘制匿名用户占比和剔除掉性别的占比图

In [51]:
from pyecharts import Pie

values1 = [len(df.loc[df['用户名'] == '匿名用户',])/len(df),1 - len(df.loc[df['用户名'] == '匿名用户',])/len(df)]
labels1 = ['匿名用户','正常用户']


values2 = [len(df.loc[df['用户名'] != '匿名用户',:].loc[df.loc[df['用户名'] != '匿名用户',:]['sex'] == '男',]),
           len(df.loc[df['sex'] == '女',]),len(df.loc[df['sex'] == '未知',])]
labels2 = ['男','女','未知']


pie = Pie('匿名情况和性别占比',title_pos = 'center',width = 1000)

pie.add('A',labels1,values1,center = [30,50],rosetype = 'radius',
        radius = [30,75],is_legend_show = False,is_label_show = True)

pie.add('B',labels2,values2,center = [75,50],rosetype = 'area',
        radius = [30,75],is_legend_show = False,is_label_show = True,
        is_random = True)

## 将时间戳转换为易读的时间格式
## 并绘制回答创建时间分布图

In [52]:
import datetime

def convert_time(x):
    return datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d')

#把更新时间，创建时间都转化成我们习惯的时间格式
df['更新时间'] = pd.to_datetime(df['更新时间'].apply(convert_time))
df['创建时间'] =pd.to_datetime(df['创建时间'].apply(convert_time))

from pyecharts import Bar

x = df.groupby('创建时间').count().index.to_list()
y = df.groupby('创建时间').count()['用户名'].to_list()

bar = Bar('回答创建时间分布',title_pos = 'center',width = 1000)
bar.add('A',x,y,is_legend_show = False,mark_point = ['max'])

## 受小Q之托，剔除掉匿名用户

In [61]:
filter_result = df.loc[df['用户名'] != '匿名用户',:]

In [62]:
filter_result.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户名,用户类型,签名,被关注人数,评论数,sex
0,POTO,<p>出于好奇，我爬取了本回答下的24620条答案，从回答中抽取了部分答主的个人信息进行了分...,2019-03-14,1,True,2019-03-23,10205,陈苟,people,"&#34;不可能不是一种事实,只是一种意见.&#34;",968,868,男
1,wang-bo-jing-85,<p>活的，母的。不要点赞，丢人。</p><p></p>,2018-10-20,1,True,2018-10-20,3709,南阳居士,people,我就是我，自己看了都上火。,758,431,男


## 第一步，筛选出不匿名的小姐姐，再筛选出回答字数在30以上的

In [64]:
#先筛选出不匿名的小姐姐
filter_result = filter_result.loc[filter_result['sex'] == '女',:]

#再筛选出回答内容大于30个字的
filter_result = filter_result.loc[filter_result['内容'].str.len() > 30,:]

print('目前小姐姐数量',len(filter_result))

目前小姐姐数量 4244


## 第二步，筛选出最近一个月的小姐姐

In [65]:
#先计算回答更新时间距离今天多少天
def delta_day(x):
    #这是在5月18日进行的筛选
    return  (datetime.datetime(2019,5,18) - x).days
filter_result['距离今日'] = filter_result['更新时间'].apply(delta_day)

#再筛选出最近一个月有更新的小姐姐
filter_result = filter_result.loc[filter_result['距离今日'] <= 30]

print('目前小姐姐数量',len(filter_result))

目前小姐姐数量 598


## 第三步，已经找到的PASS

In [66]:
#观察回答，定义一些表示找到另一半的关键字
delete_words = ['不找了','已删','删了','已经找到','已找到']

#定义一个函数，遍历关键字，如果发现就是疑似找到另一半，否则疑似正在择偶中
def filter_word(x):
    conditon = ''
    for i in delete_words:
        if x.find(i) == -1:
            condition = '疑似正在择偶中'
        else:
            return '疑似找到另一半'
    return condition
        
filter_result['择偶状态'] = filter_result['内容'].apply(filter_word)
filter_result = filter_result.loc[filter_result['择偶状态']  == '疑似正在择偶中',:]

print('目前小姐姐数量',len(filter_result))

目前小姐姐数量 562


## 第四步，关注人数1500及以上，评论数大于等于100或者点赞数在150之上的剔除

In [67]:
#知乎被关注人数超过1500
filter_result = filter_result.loc[filter_result['被关注人数'] < 1500,:]

filter_result = filter_result.loc[(filter_result['评论数'] < 100) & (filter_result['点赞数'] < 150),:]

print('目前小姐姐数量',len(filter_result))

目前小姐姐数量 480


## 引入赞评指数，用点赞数/评论数来衡量并排序

In [74]:
filter_result = filter_result.loc[(filter_result['评论数'] != 0) & (filter_result['点赞数'] != 0),:]
filter_result = filter_result.loc[filter_result['是否可以评论'] == True,:]
filter_result['赞评指数'] = filter_result['点赞数'] / filter_result['评论数']
filter_result.sort_values('赞评指数',ascending = False,inplace = True)

In [75]:
#导出大名单
filter_result.to_excel('脱单指南.xlsx')

In [70]:
len(filter_result)

402

In [ ]:
#filter_result = filter_result.loc[filter_result['点赞数'] > 10,:]